# Generative Supervised Fine-tuning of GPT-2

Now that we have our GPT-2 model all trained up - we need a way we can get it to generate what we want.

In the following notebook, we're going to use an approach called "Supervised Fine-tuning" to achieve our goals today.

In essence, we're going to use each example as a self-contained unit (with potential for something called "packing") and this is going to allow us to build "labeled" data.

For this notebook, we're going to be flying quite high up in the levels of abstraction. Take extra care to look into the libraries we're using today!

Let's start by grabbing our dependencies, as always:



In [ ]:
!pip install transformers accelerate datasets trl bitsandbytes -qU

## Dataset Curation

We're going to be fine-tuning our model on SQL generation today.

First thing we'll need is a dataset to train on!

We'll use [this](https://huggingface.co/datasets/b-mc2/sql-create-context) dataset today!

First up, let's load it and take a look at what we've got.

- [`load_dataset`](https://huggingface.co/docs/datasets/loading)

In [ ]:
from datasets import load_dataset

sql_dataset = load_dataset(
    ### YOUR CODE HERE
)

In [ ]:
sql_dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'context', 'answer'],
        num_rows: 78577
    })
})

In [ ]:
### Display a Sample Row

{'question': 'How many heads of the departments are older than 56 ?',
 'context': 'CREATE TABLE head (age INTEGER)',
 'answer': 'SELECT COUNT(*) FROM head WHERE age > 56'}

So, we've got ~78.5K rows of:

- question - a natural language query about
- context - the `CREATE TABLE` statement - which gives us important context about the table
- answer - a SQL query that is aligned with both the question and the context.

Let's split our data into `train`, `val`, and `test` datasets.

We can use our `train` and `val` sets to train and evaluate our model during training - and our `test` set to ultimately benchmark the generations of our model!

In [1]:
### YOUR CODE HERE (ensure your dataset looks similar the sample below)

In [ ]:
split_sql_dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'context', 'answer'],
        num_rows: 62861
    })
    val: Dataset({
        features: ['question', 'context', 'answer'],
        num_rows: 7858
    })
    test: Dataset({
        features: ['question', 'context', 'answer'],
        num_rows: 7858
    })
})

```
DatasetDict({
    train: Dataset({
        features: ['question', 'context', 'answer'],
        num_rows: 62861
    })
    val: Dataset({
        features: ['question', 'context', 'answer'],
        num_rows: 7858
    })
    test: Dataset({
        features: ['question', 'context', 'answer'],
        num_rows: 7858
    })
})
```

### Creating a "Prompt"

Now we need to create a prompt that's going to allow us to interact with our model when we desired the trained behaviour.

Think of this as a pattern that aligns the model with our desired outputs.

We need a single text prompt, as that is what the `SFTTrainer` we're going to use to fine-tune our model expects.

The basic idea is that we're going to merge the `question`, `context`, and `answer` into a single block of text that shows the model our desired outputs.

Let's look at what that block needs to look like:

```
{bos_token}### Instruction:
{system_message}

### Input:
{input}

### Context:
{context}

### Response:
{response}{eos_token}
```

Let's look at that from a completed prompt perspective to get a bit more information:

```
<|startoftext|>### Instruction:
You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.
You must output the SQL query that answers the question.

### Input:
How many locations did the team play at on week 7?

### Context:
CREATE TABLE table_24123547_2 (location VARCHAR, week VARCHAR)

### Response:\nSELECT COUNT(location) FROM table_24123547_2 WHERE week = 7<|endoftext|>
```

As you can see, our prompt contains completed examples of our task. We're going to show our model many of these examples over and over again to teach it to produce outputs that are aligned with our goals!

First step, let's create a template we can use to call `.format()` on while constructing our prompts.

In [ ]:
TEXT2SQL_TRAINING_PROMPT_TEMPLATE = ### CREATE A FORMATTABLE PROMPT TEMPLATE

In [ ]:
TEXT2SQL_INFERENCE_PROMPT_TEMPLATE = """\
{bos_token}### Instruction:
{system_message}

### Input:
{input}

### Context:
{context}

### Response:
"""

Now let's create a function we can map over our dataset to create the full prompt text block.

In [ ]:
def create_sql_prompt(sample):
  SYSTEM_MESSAGE = f"""You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.
  You must output the SQL query that answers the question."""

  full_prompt = TEXT2SQL_TRAINING_PROMPT_TEMPLATE.format(
      bos_token = ### YOUR CODE HERE
      eos_token = ### YOUR CODE HERE
      system_message = ### YOUR CODE HERE
      input = ### YOUR CODE HERE
      context = ### YOUR CODE HERE
      response = ### YOUR CODE HERE
  )

  return {"text" : full_prompt}

#### Helper Function Begin.

I've created this helper-function to be able to see how our model is doing visibly, rather than only through metrics.

In [ ]:
def create_sql_prompt_and_response(sample):
  SYSTEM_MESSAGE = f"""You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.
  You must output the SQL query that answers the question."""

  full_prompt = TEXT2SQL_INFERENCE_PROMPT_TEMPLATE.format(
      bos_token = "<|startoftext|>",
      system_message = SYSTEM_MESSAGE,
      input = sample["question"],
      context = sample["context"]
  )

  ground_truth = sample["answer"]

  return {"full_prompt" : full_prompt, "ground_truth" : ground_truth}

#### Helper Function End.

Let's look at an example of a formatted prompt.

In [ ]:
create_sql_prompt(split_sql_dataset["train"][0])

{'text': '<|startoftext|>### Instruction:\nYou are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.\n  You must output the SQL query that answers the question.\n\n### Input:\nName the maximum enrollment for niagara university\n\n### Context:\nCREATE TABLE table_16383772_1 (enrollment INTEGER, school VARCHAR)\n\n### Response:\nSELECT MAX(enrollment) FROM table_16383772_1 WHERE school = "Niagara University"<|endoftext|>'}

Great!

Now we can map this over our dataset!

- [`DatasetDict.map()`](https://huggingface.co/docs/datasets/process#map)

In [ ]:
split_sql_dataset = ### YOUR CODE HERE

Map:   0%|          | 0/62861 [00:00<?, ? examples/s]

Map:   0%|          | 0/7858 [00:00<?, ? examples/s]

Map:   0%|          | 0/7858 [00:00<?, ? examples/s]

## Load the Model And Preproccessing

Now for the moment we've all been waiting for...

Loading our model!

Let's use the `AutoModelForCausalLM` and `AutoTokenzier` classes from `transformers` to see just how easy this is.

- [`AutoModelForCausalLM`](https://huggingface.co/docs/transformers/v4.35.0/en/model_doc/auto#transformers.AutoModelForCausalLM)
- [`AutoTokenizer`](https://huggingface.co/docs/transformers/model_doc/auto#transformers.AutoTokenizer)
- [GPT-2 Model Card](https://huggingface.co/gpt2)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = ### YOUR MODEL ID HERE

gpt2_base_model = ### YOUR CODE HERE

gpt2_tokenizer = ### YOUR CODE HERE

We need to make sure our tokenizer has a `pad_token` in order to be able to pad sequences so they're all the same length.

We'll use a little trick here to set our padding token to our eos (end of sequence) token to make training go a little smoother.

In [ ]:
gpt2_tokenizer.pad_token = gpt2_tokenizer.eos_token

We also need to make sure we resize our model to be aligned with the token embeddings. If we didn't do this - we'd face a shape error while training!

In [ ]:
gpt2_base_model.resize_token_embeddings(len(gpt2_tokenizer))

Embedding(50257, 768)

Now let's use the Hugging Face `pipeline` to see what generation looks like for our untrained model.

In [ ]:
from transformers import pipeline, set_seed, GenerationConfig
generator = pipeline('text-generation', model=gpt2_base_model, tokenizer=gpt2_tokenizer)
set_seed(42)

def generate_sample(sample):
  prompt_package = create_sql_prompt_and_response(sample)

  generation_config = GenerationConfig(
      max_new_tokens=50,
      do_sample=True,
      top_k=50,
      temperature=1e-4,
      eos_token_id=gpt2_base_model.config.eos_token_id,
  )

  generation = generator(prompt_package["full_prompt"], generation_config=generation_config)
  print("---------------")
  print("Model Response:")
  print(generation[0]["generated_text"].replace(prompt_package["full_prompt"], ""))
  print("+++++++++++++++")
  print("Ground Truth")
  print(prompt_package["ground_truth"])

In [ ]:
generate_sample(split_sql_dataset["test"][0])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


---------------
Model Response:

You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.

### Input:

Who were the candidates where the result was
+++++++++++++++
Ground Truth
SELECT candidates FROM table_2646656_3 WHERE result = "Retired Republican hold" AND incumbent = "Philemon Bliss"


## Training the Model

Now that we have our model set up, our tokenizer set up, we can finally begin training!

Let's look at our Trainer, and set some hyper-parameters:

- `per_device_train_batch_size` - this is a batch size that accomodates distributed training - a default we could use is `4`
- `gradient_accumulation_steps` - this is exactly the same as the previous notebook, it's a way to "simulate" a large batch size by collecting losses over multiple iterations - scaling them - and then combining them together. - a default we could use is `4`
- `gradient_checkpointing` - I'll let the authors speak for themselves [here](https://github.com/cybertronai/gradient-checkpointing). In essence: This saves memory at the cost of computational time. - let's set this to `True`
- `max_grad_norm` - this is the value used for gradient clipping, which is a method of reducing vanishing gradient potential - let's use `0.3`
- `max_steps` - how many steps will we train for? - this is up to you
- `learning_rate` - how fast should we learn? - lets use `2e-4`
- `save_total_limit` - how many versions of the model will we save? - the default of `3` should work well
- `logging_steps` - how often we should log - up to you
- `output_dir` - where to save our checkpoints - up to you
- `optim` - which optimizer to use, you'll notice we're using a full precision paged optimizer - this is a performative and stable optimizer - but it uses extra memory - we should use `paged_adamw_32bit`
- `lr_scheduler_type` - we are once again using a cosine scheduler! - we should use `cosine`
- `evaluation_strategy` - we have an evaluation dataset, this defines when we should leverage it during training - we should use `steps`
- `eval_steps` - how many steps we should evaluate for - up to you
- `warmup_ration` - how many "warmup" steps we take to reach our full learning rate before we start decaying. This is a ration of our max_steps - the default value of `0.3` should work!

- [`TrainingArguments`](https://huggingface.co/docs/transformers/v4.15.0/main_classes/trainer#transformers.TrainingArguments)

In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer

training_args = TrainingArguments(
 ### YOUR CODE HERE
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


Now, for our `SFTTrainer` AKA "Where the magic happens".

This `SFTTrainer` is going to take our above training arguments, our data, our model and our tokenizer, and train it all for us!

Notice that we're setting `max_seq_length` to the maximum context window of our model - this ensures we do not exceed our maximum context window, and will pad our examples up to the maximum context window!

#### ❓QUESTION❓

What is the maximum input sequence length for GPT-2?

In [ ]:
trainer = SFTTrainer(
 gpt2_base_model,
 dataset_text_field="text",
 train_dataset=split_sql_dataset["train"],
 eval_dataset=split_sql_dataset["val"],
 tokenizer=gpt2_tokenizer,
 max_seq_length=### maximum context size of your model
 args=training_args
)

Map:   0%|          | 0/62861 [00:00<?, ? examples/s]

Map:   0%|          | 0/7858 [00:00<?, ? examples/s]

Finally, we can call our `.train()` method and watch it go!

In [ ]:
trainer.train()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
50,0.721600,0.681307
100,0.669300,0.630397
150,0.620900,0.602198
200,0.623600,0.580047
250,0.595600,0.564980
300,0.561600,0.554309
350,0.549800,0.548137
400,0.572900,0.541291
450,0.559600,0.539039
500,0.547100,0.538447


TrainOutput(global_step=500, training_loss=0.6738150663375855, metrics={'train_runtime': 729.7889, 'train_samples_per_second': 10.962, 'train_steps_per_second': 0.685, 'total_flos': 654218090496000.0, 'train_loss': 0.6738150663375855, 'epoch': 0.13})

Let's save our fine-tuned model!

In [ ]:
trainer.save_model()

## Testing our Model

Now that we have a fine-tuned model, let's see how it did

In [ ]:
ft_gpt2_model = AutoModelForCausalLM.from_pretrained("sql_gpt2")

In [ ]:
generator = pipeline('text-generation', model=ft_gpt2_model, tokenizer=gpt2_tokenizer, )

In [ ]:
generate_sample(split_sql_dataset["test"][0])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


---------------
Model Response:
SELECT candidates FROM table_2646656_3 WHERE result = "Retired Republican" AND incumbent = "Philemon Bliss" AND result = "Retired Republican" AND candidates = "Philemon Bliss" AND result = "Retired
+++++++++++++++
Ground Truth
SELECT candidates FROM table_2646656_3 WHERE result = "Retired Republican hold" AND incumbent = "Philemon Bliss"


That is *significantly* better.

#### ❓QUESTION❓

What methods could we use to validate our SQL outputs?

#### ❓QUESTION❓

How would you extend this notebook to another use-case?